In [1]:
import pandas as pd
import googlemaps
import time
import os
from dotenv import load_dotenv
from pathlib import Path


In [2]:
dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)

API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
if not API_KEY:
    raise ValueError("API Key no configurada correctamente. Verifica el archivo .env.")

# Inicializar cliente de Google Maps
gmaps = googlemaps.Client(key=API_KEY)


In [4]:
df = pd.read_csv('df_unificado_limpio2.csv')

# Eliminar coordenadas previas si existen
if 'Latitud' in df.columns and 'Longitud' in df.columns:
    df = df.drop(columns=['Latitud', 'Longitud'])


In [5]:
municipios_unicos = df[['DEPARTAMENTO', 'MUNICIPIO']].drop_duplicates()


In [6]:
df

,AÑO,TRIMESTRE,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO,CÓDIGO DANE.1,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA,VELOCIDAD SUBIDA,No. ACCESOS FIJOS A INTERNET
0,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),20.0,5.0,1
1,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),30.0,5.0,1
2,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,RESIDENCIAL - ESTRATO 1,FIBER TO THE HOME (FTTH),18.0,5.0,1
3,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5088,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),30.0,5.0,3
4,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5088,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),8.0,4.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
917685,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,2300.0,690.0,2
917686,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,4300.0,1290.0,5
917687,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,7300.0,2190.0,1
917688,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 2,WIFI,4300.0,1290.0,10


In [7]:
def obtener_coordenadas(departamento, municipio):
    try:
        query = f"{municipio}, {departamento}, Colombia"
        geocode_result = gmaps.geocode(query)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"No se encontraron coordenadas para {municipio}, {departamento}.")
            return None, None
    except Exception as e:
        print(f"Error obteniendo coordenadas para {municipio}, {departamento}: {e}")
        return None, None


In [8]:
coordenadas = []
for index, row in municipios_unicos.iterrows():
    departamento = row['DEPARTAMENTO']
    municipio = row['MUNICIPIO']

    # Obtener coordenadas
    lat, lng = obtener_coordenadas(departamento, municipio)
    coordenadas.append({'DEPARTAMENTO': departamento, 'MUNICIPIO': municipio, 'Latitud': lat, 'Longitud': lng})

    # Guardar progreso cada 10 solicitudes
    if (index + 1) % 10 == 0:
        df_progreso = pd.DataFrame(coordenadas)
        df_progreso.to_csv('Limpieza/data/progreso_coordenadas_nuevas.csv', index=False)
        print(f"Progreso guardado: {index + 1} municipios procesados.")

    # Respetar los límites de la API
    time.sleep(1)  # Ajustar para respetar la cuota


Progreso guardado: 60 municipios procesados.
Progreso guardado: 90 municipios procesados.
Progreso guardado: 100 municipios procesados.
Progreso guardado: 140 municipios procesados.
Progreso guardado: 190 municipios procesados.
Progreso guardado: 250 municipios procesados.
Progreso guardado: 260 municipios procesados.
Progreso guardado: 280 municipios procesados.
Progreso guardado: 310 municipios procesados.
Progreso guardado: 330 municipios procesados.
Progreso guardado: 340 municipios procesados.
Progreso guardado: 350 municipios procesados.
Progreso guardado: 360 municipios procesados.
Progreso guardado: 400 municipios procesados.
Progreso guardado: 410 municipios procesados.
Progreso guardado: 420 municipios procesados.
Progreso guardado: 490 municipios procesados.
Progreso guardado: 950 municipios procesados.
Progreso guardado: 1990 municipios procesados.
Progreso guardado: 2640 municipios procesados.


In [9]:
df_coordenadas_finales = pd.DataFrame(coordenadas)
df_coordenadas_finales.to_csv('Limpieza/data/coordenadas_nuevas.csv', index=False)


In [10]:
df_completo = df.merge(df_coordenadas_finales, on=['DEPARTAMENTO', 'MUNICIPIO'], how='left')
df_completo.to_csv('Limpieza/data/df_unificado_limpio_actualizado.csv', index=False)

print("Proceso completado. Coordenadas actualizadas.")


Proceso completado. Coordenadas actualizadas.
